In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GroupShuffleSplit

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

2024-11-01 12:36:11.011216: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730464571.315608    4192 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730464571.384638    4192 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-01 12:36:12.049071: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
train_data = pd.read_csv('../Data/TitanicTrain.csv')
test_data = pd.read_csv('../Data/TitanicTest.csv')

train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [8]:
train_data.shape

(891, 12)

In [10]:
X = train_data.copy().dropna()
y = X.pop('Survived')
id = X['PassengerId']

features_num = ['Age', 'SibSp', 'Parch', 'Fare']
features_cat = ['Pclass', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

preprocessor = make_column_transformer(
    (StandardScaler(), features_num),
    (OneHotEncoder(), features_cat),
)

def group_split(X, y, group, train_size=0.75):
    splitter = GroupShuffleSplit(train_size=train_size)
    train, test = next(splitter.split(X, y, groups=group))
    return (X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test])

X_train, X_valid, y_train, y_valid = group_split(X, y, id)

X_train = preprocessor.fit_transform(X_train)
X_valid = preprocessor.transform(X_valid)
y_train = y_train / 100
y_valid = y_valid / 100

input_shape = [X_train.shape[1]]
print("Input shape: {}".format(input_shape))

ValueError: Found unknown categories ['McGough, Mr. James Robert', 'Perreault, Miss. Anne', 'Sandstrom, Miss. Marguerite Rut', 'Minahan, Miss. Daisy E', 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'Swift, Mrs. Frederick Joel (Margaret Welles Barron)', 'Hogeboom, Mrs. John C (Anna Andrews)', 'Porter, Mr. Walter Chamberlain', 'Kent, Mr. Edward Austin', 'Harper, Mrs. Henry Sleeper (Myna Haxtun)', 'Silvey, Mrs. William Baird (Alice Munger)', 'Giglio, Mr. Victor', 'Francatelli, Miss. Laura Mabel', 'Hays, Mrs. Charles Melville (Clara Jennings Gregg)', 'Newell, Miss. Madeleine', 'Hoyt, Mr. Frederick Maxfield', 'Foreman, Mr. Benjamin Laventall', 'Leader, Dr. Alice (Farnham)', 'Chambers, Mr. Norman Campbell', 'Pears, Mrs. Thomas (Edith Wearne)', 'Pears, Mr. Thomas Clinton', 'Peuchen, Major. Arthur Godfrey', 'Newell, Miss. Marjorie', 'Stephenson, Mrs. Walter Bertram (Martha Eustis)', 'Davidson, Mr. Thornton', 'Troutt, Miss. Edwina Celia "Winnie"', 'Cavendish, Mr. Tyrell William', 'New

In [5]:
input_shape = [11]

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# YOUR CODE HERE
model = keras.Sequential([
    layers.BatchNormalization(),
    layers.Dense(units = 64, activation = 'relu', input_shape = input_shape),
    layers.Drop(0.3),
    layers.BatchNormalization(),
    layers.Dense(units = 64, activation = 'relu'),
    layers.Drop(0.3),
    layers.BatchNormalization(),
    layers.Dense(units = 1, activation = 'sigmoid')
])

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['binary_accuracy']
)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data = (X_valid, y_valid),
    batch_size = 32,
    epochs = 100,
    verbose = 0
)